# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 8 2022 1:36PM,239146,18,Stone-8118255,Acute Outpatient Solutions,Executing
1,Jun 8 2022 1:28PM,239145,20,ALU0007019,Alumier Labs Inc.,Released
2,Jun 8 2022 1:28PM,239145,20,ALU0007020,Alumier Labs Inc.,Released
3,Jun 8 2022 1:28PM,239145,20,ALU0007021,Alumier Labs Inc.,Released
4,Jun 8 2022 1:28PM,239145,20,ALU0007022,Alumier Labs Inc.,Released
5,Jun 8 2022 1:28PM,239145,20,ALUR113172258,Alumier Labs Inc.,Released
6,Jun 8 2022 1:28PM,239145,20,ALU0007023,Alumier Labs Inc.,Released
7,Jun 8 2022 1:27PM,239144,19,ACG-2102490276,ACG North America LLC,Released
8,Jun 8 2022 1:24PM,239143,16,AIC-375076,American International Chemical,Released
9,Jun 8 2022 1:03PM,239140,10,Enova-10102,Emerginnova,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,239140,Released,1
51,239143,Released,1
52,239144,Released,1
53,239145,Released,6
54,239146,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239140,NaN,NaN,1.0
239143,NaN,NaN,1.0
239144,NaN,NaN,1.0
239145,NaN,NaN,6.0
239146,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238998,0.0,10.0,6.0
239017,0.0,0.0,7.0
239019,29.0,3.0,2.0
239036,0.0,0.0,1.0
239039,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238998,0,10,6
239017,0,0,7
239019,29,3,2
239036,0,0,1
239039,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238998,0,10,6
1,239017,0,0,7
2,239019,29,3,2
3,239036,0,0,1
4,239039,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238998,,10,6
1,239017,,,7
2,239019,29,3,2
3,239036,,,1
4,239039,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 8 2022 1:36PM,239146,18,Acute Outpatient Solutions
1,Jun 8 2022 1:28PM,239145,20,Alumier Labs Inc.
7,Jun 8 2022 1:27PM,239144,19,ACG North America LLC
8,Jun 8 2022 1:24PM,239143,16,American International Chemical
9,Jun 8 2022 1:03PM,239140,10,Emerginnova
10,Jun 8 2022 1:00PM,239139,10,ISDIN Corporation
14,Jun 8 2022 12:35PM,239138,10,ISDIN Corporation
28,Jun 8 2022 12:34PM,239137,10,ISDIN Corporation
29,Jun 8 2022 12:26PM,239136,10,"E5 Pharma, LLC"
33,Jun 8 2022 12:22PM,239134,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 8 2022 1:36PM,239146,18,Acute Outpatient Solutions,,1,
1,Jun 8 2022 1:28PM,239145,20,Alumier Labs Inc.,,,6
2,Jun 8 2022 1:27PM,239144,19,ACG North America LLC,,,1
3,Jun 8 2022 1:24PM,239143,16,American International Chemical,,,1
4,Jun 8 2022 1:03PM,239140,10,Emerginnova,,,1
5,Jun 8 2022 1:00PM,239139,10,ISDIN Corporation,,,4
6,Jun 8 2022 12:35PM,239138,10,ISDIN Corporation,,7,7
7,Jun 8 2022 12:34PM,239137,10,ISDIN Corporation,,,1
8,Jun 8 2022 12:26PM,239136,10,"E5 Pharma, LLC",,3,1
9,Jun 8 2022 12:22PM,239134,20,"ACI Healthcare USA, Inc.",,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 1:36PM,239146,18,Acute Outpatient Solutions,,1,
1,Jun 8 2022 1:28PM,239145,20,Alumier Labs Inc.,6,,
2,Jun 8 2022 1:27PM,239144,19,ACG North America LLC,1,,
3,Jun 8 2022 1:24PM,239143,16,American International Chemical,1,,
4,Jun 8 2022 1:03PM,239140,10,Emerginnova,1,,
5,Jun 8 2022 1:00PM,239139,10,ISDIN Corporation,4,,
6,Jun 8 2022 12:35PM,239138,10,ISDIN Corporation,7,7,
7,Jun 8 2022 12:34PM,239137,10,ISDIN Corporation,1,,
8,Jun 8 2022 12:26PM,239136,10,"E5 Pharma, LLC",1,3,
9,Jun 8 2022 12:22PM,239134,20,"ACI Healthcare USA, Inc.",16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 1:36PM,239146,18,Acute Outpatient Solutions,,1,
1,Jun 8 2022 1:28PM,239145,20,Alumier Labs Inc.,6,,
2,Jun 8 2022 1:27PM,239144,19,ACG North America LLC,1,,
3,Jun 8 2022 1:24PM,239143,16,American International Chemical,1,,
4,Jun 8 2022 1:03PM,239140,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 1:36PM,239146,18,Acute Outpatient Solutions,NaN,1.0,NaN
1,Jun 8 2022 1:28PM,239145,20,Alumier Labs Inc.,6.0,NaN,NaN
2,Jun 8 2022 1:27PM,239144,19,ACG North America LLC,1.0,NaN,NaN
3,Jun 8 2022 1:24PM,239143,16,American International Chemical,1.0,NaN,NaN
4,Jun 8 2022 1:03PM,239140,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 1:36PM,239146,18,Acute Outpatient Solutions,0.0,1.0,0.0
1,Jun 8 2022 1:28PM,239145,20,Alumier Labs Inc.,6.0,0.0,0.0
2,Jun 8 2022 1:27PM,239144,19,ACG North America LLC,1.0,0.0,0.0
3,Jun 8 2022 1:24PM,239143,16,American International Chemical,1.0,0.0,0.0
4,Jun 8 2022 1:03PM,239140,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2869209,58.0,29.0,49.0
102,717254,2.0,1.0,0.0
12,239083,1.0,0.0,0.0
15,717142,9.0,11.0,25.0
16,717371,2.0,1.0,0.0
18,478253,1.0,1.0,0.0
19,956333,5.0,23.0,11.0
20,1434493,58.0,18.0,0.0
21,717200,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2869209,58.0,29.0,49.0
1,102,717254,2.0,1.0,0.0
2,12,239083,1.0,0.0,0.0
3,15,717142,9.0,11.0,25.0
4,16,717371,2.0,1.0,0.0
5,18,478253,1.0,1.0,0.0
6,19,956333,5.0,23.0,11.0
7,20,1434493,58.0,18.0,0.0
8,21,717200,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,58.0,29.0,49.0
1,102,2.0,1.0,0.0
2,12,1.0,0.0,0.0
3,15,9.0,11.0,25.0
4,16,2.0,1.0,0.0
5,18,1.0,1.0,0.0
6,19,5.0,23.0,11.0
7,20,58.0,18.0,0.0
8,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,58.0
1,102,Released,2.0
2,12,Released,1.0
3,15,Released,9.0
4,16,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,21
Status,,,,,,,,,
Completed,49.0,0.0,0.0,25.0,0.0,0.0,11.0,0.0,0.0
Executing,29.0,1.0,0.0,11.0,1.0,1.0,23.0,18.0,1.0
Released,58.0,2.0,1.0,9.0,2.0,1.0,5.0,58.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,21
0,Completed,49.0,0.0,0.0,25.0,0.0,0.0,11.0,0.0,0.0
1,Executing,29.0,1.0,0.0,11.0,1.0,1.0,23.0,18.0,1.0
2,Released,58.0,2.0,1.0,9.0,2.0,1.0,5.0,58.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,21
0,Completed,49.0,0.0,0.0,25.0,0.0,0.0,11.0,0.0,0.0
1,Executing,29.0,1.0,0.0,11.0,1.0,1.0,23.0,18.0,1.0
2,Released,58.0,2.0,1.0,9.0,2.0,1.0,5.0,58.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()